# Single QPU VQE experiment

This notebook goes through the running of a VQE algorithm using interlin-q. For simplicity, we only try out using one QPU. This notebook differs from the manual version in the manner it calculates the expectation value. Here, the controller host sends the hamiltonian to the computing hosts, and then asks it for the expectation value later through interlin-q API. This approach should scale up with the increasing number of computing hosts.

This notebook also won't include the 'single-step' implementation.

## Step 1: Import libraries.

First we import all the necessary libraries. Interlin-q is build using the python framework [QuNetSim](https://arxiv.org/abs/2003.06397) which is a python software framework that can be used to simulate quantum networks up to the network layer. We also need PennyLane's chemistry library for decomposing the Hamiltonian. We would also use PennyLane for the optimiser component

In [1]:
%load_ext autoreload

%autoreload 2

# Basic Libraries
import sys
from pennylane import numpy as np
sys.path.append("../../")

# QuNetSim Components
from qunetsim.components import Network
from qunetsim.objects import Logger
from qunetsim.backends.qutip_backend import QuTipBackend

# Interlin-q Components
from interlinq import (ControllerHost, Constants, Clock,
Circuit, Layer, ComputingHost, Operation)

# Extra needed components
from pennylane import GradientDescentOptimizer
from hamiltonian_decomposition import decompose

Logger.DISABLED = False

## Step 2: Decompose the Hamiltonian.

In [2]:
geometry = 'h2.xyz'
charge = 0
multiplicity = 1
basis_set = 'sto-3g'
name = 'h2'

In [3]:
coefficients, observables, qubit_num = decompose(name, geometry, charge, multiplicity, basis_set)
terms = list(zip(coefficients, observables))

terms

[(-0.04207897647782276, [('Identity', 0)]),
 (0.17771287465139946, [('PauliZ', 0)]),
 (0.1777128746513994, [('PauliZ', 1)]),
 (-0.24274280513140462, [('PauliZ', 2)]),
 (-0.24274280513140462, [('PauliZ', 3)]),
 (0.17059738328801052, [('PauliZ', 0), ('PauliZ', 1)]),
 (0.04475014401535161,
  [('PauliY', 0), ('PauliX', 1), ('PauliX', 2), ('PauliY', 3)]),
 (-0.04475014401535161,
  [('PauliY', 0), ('PauliY', 1), ('PauliX', 2), ('PauliX', 3)]),
 (-0.04475014401535161,
  [('PauliX', 0), ('PauliX', 1), ('PauliY', 2), ('PauliY', 3)]),
 (0.04475014401535161,
  [('PauliX', 0), ('PauliY', 1), ('PauliY', 2), ('PauliX', 3)]),
 (0.12293305056183798, [('PauliZ', 0), ('PauliZ', 2)]),
 (0.1676831945771896, [('PauliZ', 0), ('PauliZ', 3)]),
 (0.1676831945771896, [('PauliZ', 1), ('PauliZ', 2)]),
 (0.12293305056183798, [('PauliZ', 1), ('PauliZ', 3)]),
 (0.17627640804319591, [('PauliZ', 2), ('PauliZ', 3)])]

## Step 3: Prepare Circuit for Given Parameters

The circuit can be prepared using two different ways: either as one circuit, or several circuits run sequentially. The former approach is simpler and generally better for the optimisation function. The latter is better for debugging and for dynamic components of a quantum circuit (i.e. circuits that have a lot of changing operations).

### Main Blocks

In [4]:
def rotational_gate(params):
    phi, theta, omega = params
    cos = np.cos(theta / 2)
    sin = np.sin(theta / 2)
    
    res = np.array([[np.exp(-1j * (phi + omega) / 2) * cos, -np.exp(1j * (phi - omega) / 2) * sin], 
                     [np.exp(-1j * (phi - omega) / 2) * sin, np.exp(1j * (phi + omega) / 2) * cos]])
    
    return res.unwrap()

In [5]:
def initialisation_operations(q_map):
    ops = []
    host_id = list(q_map.keys())[0]
    
    op = Operation(
        name=Constants.PREPARE_QUBITS,
        qids=q_map[host_id],
        computing_host_ids=[host_id])
    ops.append(op)
    
    # Prepare the qubits on the computing host
    op = Operation(
        name=Constants.SINGLE,
        qids=[q_map[host_id][0]],
        gate=Operation.X,
        computing_host_ids=[host_id])
    ops.append(op)
    
    op = Operation(
        name=Constants.SINGLE,
        qids=[q_map[host_id][1]],
        gate=Operation.X,
        computing_host_ids=[host_id])
    ops.append(op)
    
    return Layer(ops)

In [6]:
def ansatz_operations(q_map, parameters):
    layers = []
    host_id = list(q_map.keys())[0]
    
    ops = []
    
    for i in range(len(q_map[host_id])):
        op = Operation(
            name=Constants.SINGLE,
            qids=[q_map[host_id][i]],
            gate=Operation.CUSTOM,
            gate_param=rotational_gate(parameters[i]),
            computing_host_ids=[host_id])
        
        ops.append(op)
        
    layers.append(Layer(ops))
    
    op = Operation(
        name=Constants.TWO_QUBIT,
        qids=[q_map[host_id][2], q_map[host_id][3]],
        gate=Operation.CNOT,
        computing_host_ids=[host_id])
    
    layers.append(Layer([op]))
    
    op = Operation(
        name=Constants.TWO_QUBIT,
        qids=[q_map[host_id][2], q_map[host_id][0]],
        gate=Operation.CNOT,
        computing_host_ids=[host_id])
    
    layers.append(Layer([op]))
    
    op = Operation(
        name=Constants.TWO_QUBIT,
        qids=[q_map[host_id][3], q_map[host_id][1]],
        gate=Operation.CNOT,
        computing_host_ids=[host_id])
    
    layers.append(Layer([op]))
        
    return layers

In [7]:
def measurement_operations(q_map):
    ops = []
    # Measuring only the first qubit
    op = Operation(
        name=Constants.MEASURE,
        qids=['q_0_0'],
        cids=['q_0_0'],
        computing_host_ids=[host_id])
    ops.append(op)
    layers.append(Layer(ops))
    
    # Measuring all qubits
    q_ids = q_map[host_id].copy()
    ops = []
    for q_id in q_ids:
        op = Operation(
            name=Constants.MEASURE,
            qids=[q_id],
            cids=[q_id],
            computing_host_ids=[computing_host_ids[0]])
        ops.append(op)
        
    return Layer(ops)

### The Protocols

#### The Controller Protocols 

In [8]:
def prepare_qubits(q_map): 
    circuit = Circuit(q_map, [initialisation_operations(q_map)])
    return circuit

In [9]:
def apply_ansatz(q_map, parameters):
    circuit = Circuit(q_map, ansatz_operations(q_map, parameters))
    return circuit

In [10]:
def controller_host_protocol_preparation(host, q_map, params):
    """
    Protocol for the controller host
    """
    host.generate_and_send_schedules(prepare_qubits(q_map))

In [11]:
def controller_host_protocol_ansatz(host, q_map, params):
    """
    Protocol for the controller host
    """
    host.generate_and_send_schedules(apply_ansatz(q_map, params))

In [12]:
def controller_host_protocol_expectation_schedule(host, q_map, terms):
    """
    Protocol for the controller host
    """
    host.schedule_expectation_terms(terms, q_map)
    
    host.dispatch_hamiltonian_schedules(q_map)

In [13]:
def controller_host_protocol_expectation_collection(host, q_map, terms):
    """
    Protocol for the controller host
    """
    host.ask_for_expectation_values(q_map)
    
    host.receive_results()

#### The Computing Protocols

In [14]:
def computing_host_protocol(host):
    host.receive_schedule()

In [15]:
def computing_host_protocol_send_results(host):
    host.receive_schedule()
    
    host.send_results('expectation')

## Step 4: Run the circuit and get the Expectation Value

In [15]:
def init_network():
    network = Network.get_instance()
    network.delay = 0
    network.start()

    clock = Clock()

    qutip = QuTipBackend()

    controller_host = ControllerHost(
        host_id="host_1",
        clock=clock, 
        backend=qutip
    )

    computing_hosts, q_map = controller_host.create_distributed_network(
        num_computing_hosts=1,
        num_qubits_per_host=4)
    controller_host.start()

    network.add_hosts([controller_host])
    
    network.add_hosts(computing_hosts)
    
    return clock, controller_host, computing_hosts, q_map

In [16]:
params = np.random.rand(4, 3)

### Running the circuit

In [17]:
clock, controller_host, computing_hosts, q_map = init_network()

INFO:qu_net_sim:Host QPU_0 started processing
INFO:qu_net_sim:Host host_1 started processing


In [18]:
t1 = controller_host.run_protocol(
    controller_host_protocol_preparation,
    (q_map, params))
t2 = computing_hosts[0].run_protocol(computing_host_protocol)

t1.join()
t2.join()

INFO:qu_net_sim:host_1 sends BROADCAST message
INFO:qu_net_sim:sending ACK:1 from QPU_0 to host_1
INFO:qu_net_sim:QPU_0 received {"QPU_0": [{"name": "PREPARE_QUBITS", "qids": ["q_0_0", "q_0_1", "q_0_2", "q_0_3"], "cids": null, "gate": null, "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "SINGLE", "qids": ["q_0_0"], "cids": null, "gate": "X", "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "SINGLE", "qids": ["q_0_1"], "cids": null, "gate": "X", "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}]} with sequence number 0
INFO:qu_net_sim:QPU_0 sends CLASSICAL to host_1 with sequence 0
INFO:qu_net_sim:host_1 received ACK from QPU_0 with sequence number 0
INFO:qu_net_sim:QPU_0 awaits classical ACK from host_1 with sequence 0
INFO:qu_net_sim:sending ACK:1 from host_1 to QPU_0
INFO:qu_net_sim:host_1 received ACK with seque

In [19]:
clock.ticks

2

In [20]:
t1 = controller_host.run_protocol(
    controller_host_protocol_ansatz,
    (q_map, params))
t2 = computing_hosts[0].run_protocol(computing_host_protocol)

t1.join()
t2.join()

INFO:qu_net_sim:host_1 sends BROADCAST message
INFO:qu_net_sim:sending ACK:2 from QPU_0 to host_1
INFO:qu_net_sim:QPU_0 received {"QPU_0": [{"name": "SINGLE", "qids": ["q_0_0"], "cids": null, "gate": "custom_gate", "gate_param": [[[0.7797332733245818, -0.5454497675834412], [-0.29534300451558604, 0.08528237331013755]], [[0.29534300451558604, 0.08528237331013755], [0.7797332733245818, 0.5454497675834412]]], "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 2}, {"name": "SINGLE", "qids": ["q_0_1"], "cids": null, "gate": "custom_gate", "gate_param": [[[0.9154102304549301, -0.37478225442009233], [-0.13921115392080424, 0.04671858703217794]], [[0.13921115392080424, 0.04671858703217794], [0.9154102304549301, 0.37478225442009233]]], "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 2}, {"name": "SINGLE", "qids": ["q_0_2"], "cids": null, "gate": "custom_gate", "gate_param": [[[0.7603881644935007, -0.6162734967135605], [-0.2001219158402687, -0

In [21]:
clock.ticks

7

In [22]:
t1 = controller_host.run_protocol(
    controller_host_protocol_expectation_schedule,
    (q_map, terms))
t2 = computing_hosts[0].run_protocol(computing_host_protocol)

t1.join()
t2.join()

INFO:qu_net_sim:host_1 sends BROADCAST message
INFO:qu_net_sim:sending ACK:3 from QPU_0 to host_1
INFO:qu_net_sim:QPU_0 received {"QPU_0": [{"name": "REC_HAMILTON", "qids": null, "cids": null, "gate": null, "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "hamiltonian": [[-0.04207897647782276, [["Identity", 0]]], [0.17771287465139946, [["PauliZ", 0]]], [0.1777128746513994, [["PauliZ", 1]]], [-0.24274280513140462, [["PauliZ", 2]]], [-0.24274280513140462, [["PauliZ", 3]]], [0.17059738328801052, [["PauliZ", 0], ["PauliZ", 1]]], [0.04475014401535161, [["PauliY", 0], ["PauliX", 1], ["PauliX", 2], ["PauliY", 3]]], [-0.04475014401535161, [["PauliY", 0], ["PauliY", 1], ["PauliX", 2], ["PauliX", 3]]], [-0.04475014401535161, [["PauliX", 0], ["PauliX", 1], ["PauliY", 2], ["PauliY", 3]]], [0.04475014401535161, [["PauliX", 0], ["PauliY", 1], ["PauliY", 2], ["PauliX", 3]]], [0.12293305056183798, [["PauliZ", 0], ["PauliZ", 2]]], [0.1676831945771896, [["PauliZ", 0], 

In [23]:
clock.ticks

9

In [ ]:
t1 = controller_host.run_protocol(
    controller_host_protocol_expectation_collection,
    (q_map, terms))
t2 = computing_hosts[0].run_protocol(computing_host_protocol_send_results)

t1.join()
t2.join()

INFO:qu_net_sim:host_1 sends BROADCAST message
INFO:qu_net_sim:sending ACK:4 from QPU_0 to host_1
INFO:qu_net_sim:QPU_0 received {"QPU_0": [{"name": "SEND_EXP", "qids": null, "cids": null, "gate": null, "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 9}]} with sequence number 3
INFO:qu_net_sim:host_1 received ACK from QPU_0 with sequence number 3
INFO:qu_net_sim:QPU_0 sends CLASSICAL to host_1 with sequence 3
INFO:qu_net_sim:QPU_0 awaits classical ACK from host_1 with sequence 3
INFO:qu_net_sim:sending ACK:4 from host_1 to QPU_0
INFO:qu_net_sim:host_1 received ACK with sequence number 3
INFO:qu_net_sim:QPU_0 received ACK from host_1 with sequence number 3


In [26]:
clock.ticks

11

## Optimise

In [33]:
def cost_fn(params):
    params = params.reshape(4, 3)
    
    network = Network.get_instance()
    network.delay = 0
    network.start()

    clock = Clock()

    qutip = QuTipBackend()

    controller_host = ControllerHost(
        host_id="host_1",
        clock=clock, 
        backend=qutip
    )

    computing_hosts, q_map = controller_host.create_distributed_network(
        num_computing_hosts=1,
        num_qubits_per_host=4)
    controller_host.start()

    network.add_hosts([
        computing_hosts[0],
        controller_host])
    
    t1 = controller_host.run_protocol(controller_host_protocol_preparation, (q_map, params))
    t2 = computing_hosts[0].run_protocol(computing_host_protocol)

    t1.join()
    t2.join()
    
    t1 = controller_host.run_protocol(controller_host_protocol_ansatz, (q_map, params))
    t2 = computing_hosts[0].run_protocol(computing_host_protocol)

    t1.join()
    t2.join()
    
    t1 = controller_host.run_protocol(controller_host_protocol_expectation_schedule, (q_map, terms))
    t2 = computing_hosts[0].run_protocol(computing_host_protocol)

    t1.join()
    t2.join()
        
    t1 = controller_host.run_protocol(controller_host_protocol_expectation_collection, (q_map, terms))
    t2 = computing_hosts[0].run_protocol(computing_host_protocol_send_results)

    t1.join()
    t2.join()

    network.remove_host(computing_hosts[0])
    network.remove_host(controller_host)
    
    total_exp = 0
    for host in controller_host.computing_host_ids:
        total_exp += controller_host.results[host]['bits']
    
    return total_exp

In [28]:
# np.random.seed(0)
params = np.random.normal(0, np.pi, (4, 3))

params

tensor([[ 1.02025713,  2.75143507,  0.35108503],
        [-5.90293328, -0.499001  ,  1.29220997],
        [-0.73486447,  0.27990438,  1.26653549],
        [ 4.59111796,  2.23811977, -5.28321501]], requires_grad=True)

In [29]:
cost_fn(params)

INFO:qu_net_sim:Host QPU_0 started processing
INFO:qu_net_sim:Host host_1 started processing
INFO:qu_net_sim:host_1 sends BROADCAST message
INFO:qu_net_sim:sending ACK:1 from QPU_0 to host_1
INFO:qu_net_sim:QPU_0 received {"QPU_0": [{"name": "PREPARE_QUBITS", "qids": ["q_0_0", "q_0_1", "q_0_2", "q_0_3"], "cids": null, "gate": null, "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "SINGLE", "qids": ["q_0_0"], "cids": null, "gate": "X", "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "SINGLE", "qids": ["q_0_1"], "cids": null, "gate": "X", "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}]} with sequence number 0
INFO:qu_net_sim:QPU_0 sends CLASSICAL to host_1 with sequence 0
INFO:qu_net_sim:host_1 received ACK from QPU_0 with sequence number 0
INFO:qu_net_sim:QPU_0 awaits classical ACK from host_1 with sequence 0
INFO

here


INFO:qu_net_sim:sending ACK:2 from QPU_0 to host_1
INFO:qu_net_sim:QPU_0 received {"QPU_0": [{"name": "SINGLE", "qids": ["q_0_0"], "cids": null, "gate": "custom_gate", "gate_param": [[[0.15003402056456178, -0.12274047369376546], [-0.9266305457792057, -0.32214965531503265]], [[0.9266305457792057, -0.32214965531503265], [0.15003402056456178, 0.12274047369376546]]], "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 2}, {"name": "SINGLE", "qids": ["q_0_1"], "cids": null, "gate": "custom_gate", "gate_param": [[[-0.6495104830710016, 0.7191430088320073], [-0.22169223831714102, 0.10872909775610923]], [[0.22169223831714102, 0.10872909775610923], [-0.6495104830710016, -0.7191430088320073]]], "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 2}, {"name": "SINGLE", "qids": ["q_0_2"], "cids": null, "gate": "custom_gate", "gate_param": [[[0.9554394479857654, -0.2601468641479068], [-0.07528770407574095, 0.11743437282870041]], [[0.07528770407574095

0.8509328247483128

In [30]:
Logger.DISABLED = True

### SciPy Optimisers

In [31]:
from scipy.optimize import minimize
minimum_bfgs = minimize(cost_fn, params, method="BFGS", tol=1e-4)

minimum_bfgs

here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here


      fun: -1.4585438011944087
 hess_inv: array([[ 1.00000000e+00,  7.31877655e-07,  3.36177045e-12,
         3.53135002e-12, -1.06938794e-06,  3.51985157e-12,
        -2.41685467e-12, -6.05781090e-07, -2.41685467e-12,
        -5.85385239e-13,  5.34605201e-07, -5.60978918e-13],
       [ 7.31877655e-07,  1.58289426e+00,  7.04610575e-07,
         9.39402238e-07, -1.82778972e-02,  9.30713117e-07,
         3.92140213e-07,  1.35605677e-02,  3.92140213e-07,
         1.40924663e-07, -2.06039583e-02,  1.64494210e-07],
       [ 3.36177045e-12,  7.04610575e-07,  1.00000000e+00,
         3.49792993e-12, -1.04060305e-06,  3.48684807e-12,
        -2.41541428e-12, -6.03337570e-07, -2.41541428e-12,
        -5.85170365e-13,  5.38585721e-07, -5.60831909e-13],
       [ 3.53135002e-12,  9.39402238e-07,  3.49792993e-12,
         1.00000000e+00, -1.21774218e-06,  3.68336952e-12,
        -2.41501231e-12, -5.55136367e-07, -2.41501231e-12,
        -6.05045170e-13,  5.46340644e-07, -5.77009024e-13],
       [-1

In [32]:
minimum_bfgs.x.reshape(4,3)

array([[ 1.02025731e+00, -1.12461219e-05,  3.51085249e-01],
       [-5.90293337e+00, -2.83175206e-05,  1.29220990e+00],
       [-7.34864514e-01, -7.50651418e-06,  1.26653544e+00],
       [ 4.59111810e+00, -1.70873340e-05, -5.28321502e+00]])

In [64]:
from scipy.optimize import minimize
minimum_powell = minimize(cost_fn, params, method="Powell", tol=1e-4)

minimum_powell

In [35]:
minimum_powell.x.reshape(4,3)

array([[ 8.64960745e+00, -3.37645419e-06,  6.50184703e+00],
       [ 1.04498037e+01,  6.28318531e+00,  4.49597326e-01],
       [ 4.05775526e+00, -8.15788377e-11,  2.26365738e+00],
       [ 6.37968963e-01,  2.44609655e-10,  6.60317682e+00]])

In [34]:
from scipy.optimize import minimize
minimum_cobyla = minimize(cost_fn, params, method="COBYLA", tol=1e-4)

minimum_cobyla

Exception in thread Thread-11357:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-15-902721c51af7>", line 4, in computing_host_protocol_send_results
  File "../../interlinq/components/computing_host.py", line 645, in send_results
    'bits': self.exp
AttributeError: 'ComputingHost' object has no attribute 'exp'


KeyboardInterrupt: 

In [37]:
minimum_cobyla.x.reshape(4,3)

array([[ 5.55694514e+00, -7.61630109e-06,  4.28383432e+00],
       [ 6.28318768e+00,  5.80744365e+00, -3.31610951e+00],
       [ 2.96686515e+00, -1.04722187e+00,  1.59533893e-05],
       [ 9.63660043e-01,  4.09808099e-05,  4.04340722e+00]])

### Scikit-Quant Optimisers

In [35]:
from skquant.interop.scipy import *

In [36]:
bounds = np.array([-3*np.pi, 3*np.pi])
bounds = np.tile(bounds, (4*3, 1))

flattened_parameters = params.flatten()

In [39]:
budget = 200
minimum_imfil = minimize(cost_fn, flattened_parameters, method=imfil, bounds=bounds, options={'budget' : budget})

minimum_imfil

     fun: -1.3256110073898835
 message: 'completed'
    nfev: 184
  status: 0
 success: True
       x: array([ 1.02025713, -9.42477796,  0.35108503, -5.90293328,  0.        ,
        1.29220997, -0.73486447, -9.42477796,  1.26653549,  4.59111796,
       -9.42477796, -5.28321501])

In [38]:
budget = 100
minimum_bobyqa = minimize(cost_fn, flattened_parameters, method=pybobyqa, bounds=bounds, options={'budget' : budget})

minimum_bobyqa

     fun: -1.0362649459088442
 message: 'completed'
    nfev: 100
  status: 0
 success: True
       x: array([ 1.64397148,  0.72117066, -0.27262932, -5.27921892, -0.9354028 ,
        1.91592432, -0.11115012,  0.33157929,  1.89024984,  5.21483232,
        0.24504781, -4.65950066])

In [61]:
budget = 200
minimum_snobfit = minimize(cost_fn, flattened_parameters, method=snobfit, bounds=bounds, options={'budget' : budget})

minimum_snobfit

     fun: -1.3371320476152808
 message: 'completed'
    nfev: 211
  status: 0
 success: True
       x: array([-5.50011192,  0.33891502, -4.96874294,  4.34972352, -6.29650566,
       -4.42851467,  1.59863084, -5.96739241, -3.51657315,  3.89921914,
       -6.50648971, -6.01809772])

### Gradient Descent Optimisers 

In [38]:
%debug
#max_iterations = 200
#conv_tol = 1e-06

#for n in range(max_iterations):
#    params, prev_energy = opt.step_and_cost(cost_fn, params)
#    energy = cost_fn(params)
#    conv = np.abs(energy - prev_energy)

#    if n % 20 == 0:
#        print('Iteration = {:},  Energy = {:.8f} Ha'.format(n, energy))

#    if conv <= conv_tol:
#        break

ERROR:root:No traceback has been produced, nothing to debug.
